In [8]:
data_file = "./data/AMAZON_FASHION.json"
save_folder = "./data/fashion"
f = open(data_file, "r", encoding = "utf-8")
a = f.readline()
f.close()
a = json.loads(a)
print(a['reviewText'])

Exactly what I needed.


In [2]:
import os
import csv
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

In [2]:
from BertCoNN.utils import DeepCoNNDataset, predict_mse, date

In [6]:
train_dataset = DeepCoNNDataset("./data/fashion/rating_5core_train.csv")
train_dlr = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [7]:
for batch in train_dlr:
    users, items, ratings = batch
    print(users)
    break

('A4GR8SDFRBSDV', 'A37Y0PJNGMVNJK', 'AV2MXVWMTERBK', 'A3L65LU0YQNQ36', 'A1LZXPCNQZ7BCF', 'A1PZDJFFV6O0MS', 'A21ZV3Q9IPI0VL', 'A32GWIG35XVBT5', 'A2FOBKHRMBPC8E', 'A3W5A9X3HHACQT')


In [ ]:
os.environ['TRANSFORMERS_CACHE'] = '/home/yenncs/cache/'
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [13]:
sentences = [
    'This is the first sentence.',
    'This is the second sentence.',
    'This is the third sentence.',
    'This is the fourth sentence.',
    'This is the fifth sentence.',
    'This is the sixth sentence.',
    'This is the seventh sentence.',
    'This is the eighth sentence.',
    'This is the ninth sentence.',
    'This is the tenth sentence.'
]
encoded_inputs = tokenizer.batch_encode_plus(sentences, padding=True, return_tensors='pt')
with torch.no_grad():
    model_outputs = model(**encoded_inputs)
    last_hidden_states = model_outputs.last_hidden_state
sentence_embeddings = torch.mean(last_hidden_states, dim=1).squeeze()


In [ ]:
sen_model = SentenceTransformer('bert-large-nli-mean-tokens')

In [10]:
print("Max Sequence Length:", sen_model.max_seq_length)

Max Sequence Length: 128


In [13]:
sen_model.max_seq_length = 2000
print("Max Sequence Length:", sen_model.max_seq_length)

Max Sequence Length: 2000


In [16]:
sentences = ['This is an example sentence.', 'Each sentence is converted to a fixed-length vector.']
embeddings = sen_model.encode(sentences)

In [17]:
embeddings.shape

(2, 1024)

In [20]:
from transformers import AutoTokenizer, AutoModel
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-large-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-large-nli-mean-tokens')

In [43]:
sentences = ['''Vietnam[b] (Vietnamese: Việt Nam, [vîət nāːm] (listen), commonly abbreviated VN), officially the Socialist Republic of Vietnam (SRV),[c] is a country in Southeast Asia. It is located at the eastern edge of mainland Southeast Asia, with an area of 331,212 square kilometres (127,882 sq mi) and population of 99 million, making it the world's fifteenth-most populous country. Vietnam share land borders with China to the north, and Laos and Cambodia to the west. It shares maritime borders with Thailand through the Gulf of Thailand, and the Philippines, Indonesia, and Malaysia through the South China Sea. Its capital is Hanoi and its largest city is Ho Chi Minh City (commonly referred to by its former name, Saigon).

Vietnam was inhabited by the Paleolithic age, with states established in the first millennium BC on the Red River Delta in modern-day northern Vietnam. The Han dynasty annexed Northern and Central Vietnam under Chinese rule from 111 BC, until the first dynasty emerged in 939. Successive monarchical dynasties absorbed Chinese influences through Confucianism and Buddhism, and expanded southward to the Mekong Delta, conquering Champa. The Nguyễn—the last imperial dynasty—surrendered to France in 1883. Following the August Revolution, the nationalist coalition Viet Minh under the leadership of communist revolutionary Ho Chi Minh proclaimed independence of Vietnam in 1945.

Vietnam went through prolonged warfare in the 20th century. After World War II, France returned to reclaim colonial power in the First Indochina War, from which Vietnam emerged victorious in 1954. As a result of the treaties signed between the Viet Minh and France, Vietnam was also separated into two parts. The Vietnam War began shortly after, between the communist North, supported by the Soviet Union and China, and the anti-communist South, supported by the United States. Upon the North Vietnamese victory in 1975, Vietnam reunified as a unitary socialist state under the Communist Party of Vietnam (CPV) in 1976. An ineffective planned economy, a trade embargo by the West, and wars with Cambodia and China crippled the country further. In 1986, the CPV initiated economic and political reforms similar to the Chinese economic reform, transforming the country to a market-oriented economy. The reforms facilitated Vietnamese reintegration into the global economy and politics.

A developing country with a lower-middle-income economy, Vietnam is nonetheless one of the fastest-growing economies of the 21st century, with a GDP predicted to rival developed nations by 2050. Vietnam has high levels of corruption and censorship and a poor human rights record; the country ranks among the lowest in international measurements of civil liberties, freedom of the press, and freedom of religion and ethnic minorities. It is part of international and intergovernmental institutions including the ASEAN, the APEC, the CPTPP, the Non-Aligned Movement, the OIF, and the WTO. It has assumed a seat on the United Nations Security Council twice.

Etymology
Main article: Names of Vietnam
The name Việt Nam (Vietnamese pronunciation: [viə̀t naːm], chữ Hán: 越南), literally “Viet South”, means “Viet of the South” per Vietnamese word order or “South of the Viet” per Classical Chinese word order.[9] A variation of the name, Nanyue (or Nam Việt, 南越), was first documented in the 2nd century BC.[10] The term "Việt" (Yue) (Chinese: 越; pinyin: Yuè; Cantonese Yale: Yuht; Wade–Giles: Yüeh4; Vietnamese: Việt) in Early Middle Chinese was first written using the logograph "戉" for an axe (a homophone), in oracle bone and bronze inscriptions of the late Shang dynasty (c. 1200 BC), and later as "越".[11] At that time it referred to a people or chieftain to the northwest of the Shang.[12] In the early 8th century BC, a tribe on the middle Yangtze were called the Yangyue, a term later used for peoples further south.[12] Between the 7th and 4th centuries BC Yue/Việt referred to the State of Yue in the lower Yangtze basin and its people.[11][12] From the 3rd century BC the term was used for the non-Chinese populations of southern China and northern Vietnam, with particular ethnic groups called Minyue, Ouyue, Luoyue (Vietnamese: Lạc Việt), etc., collectively called the Baiyue (Bách Việt, Chinese: 百越; pinyin: Bǎiyuè; Cantonese Yale: Baak Yuet; Vietnamese: Bách Việt; "Hundred Yue/Viet").[11][12][13] The term Baiyue/Bách Việt first appeared in the book Lüshi Chunqiu compiled around 239 BC.[14] By the 17th and 18th centuries AD, educated Vietnamese apparently referred to themselves as nguoi Viet (Viet people) or nguoi nam (southern people).[15]

The form Việt Nam (越南) is first recorded in the 16th-century oracular poem Sấm Trạng Trình. The name has also been found on 12 steles carved in the 16th and 17th centuries, including one at Bao Lam Pagoda in Hải Phòng that dates to 1558.[16] In 1802, Nguyễn Phúc Ánh (who later became Emperor Gia Long) established the Nguyễn dynasty. In the second year of his rule, he asked the Jiaqing Emperor of the Qing dynasty to confer on him the title 'King of Nam Việt / Nanyue' (南越 in Chinese character) after seizing power in Annam. The Emperor refused because the name was related to Zhao Tuo's Nanyue, which included the regions of Guangxi and Guangdong in southern China. The Qing Emperor, therefore, decided to call the area "Việt Nam" instead,[d][18] meaning “South of the Viet” per Classical Chinese word order but the Vietnamese understood it as “Viet of the South” per Vietnamese word order.[9] Between 1804 and 1813, the name Vietnam was used officially by Emperor Gia Long.[d] It was revived in the early 20th century in Phan Bội Châu's History of the Loss of Vietnam, and later by the Vietnamese Nationalist Party (VNQDĐ).[19] The country was usually called Annam until 1945, when the imperial government in Huế adopted Việt Nam.[20]

History
Main article: History of Vietnam
For a chronological guide, see Timeline of Vietnamese history.
Prehistory and early history
Photograph of a Đông Sơn bronze drum
A Đông Sơn bronze drum, c. 800 BC
Archaeological excavations have revealed the existence of humans in what is now Vietnam as early as the Paleolithic age. Stone artefacts excavated in Gia Lai province have been claimed to date to 0.78 Ma,[21] based on associated find of tektites, however this claim has been challenged because tektites are often found in archaeological sites of various ages in Vietnam.[22] Homo erectus fossils dating to around 500,000 BC have been found in caves in Lạng Sơn and Nghệ An provinces in northern Vietnam.[23] The oldest Homo sapiens fossils from mainland Southeast Asia are of Middle Pleistocene provenance, and include isolated tooth fragments from Tham Om and Hang Hum.[24][25][26] Teeth attributed to Homo sapiens from the Late Pleistocene have been found at Dong Can,[27] and from the Early Holocene at Mai Da Dieu,[28][29] Lang Gao[30][31] and Lang Cuom.[32] By about 1,000 BC, the development of wet-rice cultivation in the Ma River and Red River floodplains led to the flourishing of Đông Sơn culture,[33][34] notable for its bronze casting used to make elaborate bronze Đông Sơn drums.[35][36][37] At this point, the early Vietnamese kingdoms of Văn Lang and Âu Lạc appeared, and the culture's influence spread to other parts of Southeast Asia, including Maritime Southeast Asia, throughout the first millennium BC.[36][38]''']

In [32]:
sen_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-large-nli-mean-tokens')

In [33]:
encoded_input = sen_tokenizer(sentences, padding=True, return_tensors='pt')

In [36]:
out_sentence = sen_tokenizer.batch_decode(encoded_input['input_ids'], skip_special_tokens=True)

In [37]:
print(out_sentence)

['vietnam [ b ] ( vietnamese : viet nam, [ viət naːm ] ( listen ), commonly abbreviated vn ), officially the socialist republic of vietnam ( srv ), [ c ] is a country in southeast asia. it is located at the eastern edge of mainland southeast asia, with an area of 331, 212 square kilometres ( 127, 882 sq mi ) and population of 99 million, making it the world\'s fifteenth - most populous country. vietnam share land borders with china to the north, and laos and cambodia to the west. it shares maritime borders with thailand through the gulf of thailand, and the philippines, indonesia, and malaysia through the south china sea. its capital is hanoi and its largest city is ho chi minh city ( commonly referred to by its former name, saigon ). vietnam was inhabited by the paleolithic age, with states established in the first millennium bc on the red river delta in modern - day northern vietnam. the han dynasty annexed northern and central vietnam under chinese rule from 111 bc, until the first 

In [45]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-large-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-large-nli-mean-tokens')

sentences = ['This is an example sentence', 'Each sentence is converted']
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings:
tensor([[-0.6649, -0.0894,  0.7445,  ...,  0.3436, -0.2317,  0.4688],
        [-0.1336, -0.5473,  0.2303,  ..., -0.5988, -0.6033,  0.2249]])


In [46]:
sentence_embeddings.shape

torch.Size([2, 1024])

In [48]:
encoded_input

{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}